# 추론: 배포한 공유모델로 객체감지

In [2]:
import torch
print("torch:", 
      torch.__version__, 
      "cuda:", torch.version.cuda, 
      "is_available:", 
      torch.cuda.is_available())

torch: 2.6.0+cu124 cuda: 12.4 is_available: True


In [3]:
import os
from dotenv import load_dotenv
loaded = load_dotenv()
# os.getenv('ROBOFLOW_API_KEY')

In [4]:
from inference_sdk import InferenceHTTPClient
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key = os.getenv('ROBOFLOW_API_KEY')
)
CLIENT

In [5]:
result = CLIENT.infer('sample.png', model_id="rock-paper-scissors-sxsw/14")
result

{'inference_id': '9b9c6d78-4422-4c12-9e08-961ab55aa21f',
 'time': 0.049025673999949504,
 'image': {'width': 580, 'height': 614},
 'predictions': [{'x': 129.5,
   'y': 119.0,
   'width': 257.0,
   'height': 96.0,
   'confidence': 0.8729892373085022,
   'class': 'Scissors',
   'class_id': 2,
   'detection_id': '91270045-68af-40ef-8bb6-e1b3b93b49b8'},
  {'x': 117.0,
   'y': 316.5,
   'width': 230.0,
   'height': 123.0,
   'confidence': 0.7074464559555054,
   'class': 'Paper',
   'class_id': 0,
   'detection_id': '1233dca9-fbce-4f82-abd6-ec66eff3544d'},
  {'x': 128.0,
   'y': 502.5,
   'width': 254.0,
   'height': 109.0,
   'confidence': 0.4878521263599396,
   'class': 'Paper',
   'class_id': 0,
   'detection_id': 'cd359ed8-a6ea-435c-8d38-601cef443b0c'}]}

In [ ]:
import cv2
img = cv2.imread('sample.png')

for pred in result['predictions']:
    x = pred['x']
    y = pred['y']
    width = pred['width']
    height = pred['height']
    conf = pred['confidence']
    obj_class = pred['class']
    
    x1, y1 = int(x-width/2), int(y-height/2)
    x2, y2 = int(x+width/2), int(y+height/2)
    cv2.rectangle(img, (x1,y1), (x2, y2), (0,0,255), 2)
    cv2.putText(img, f'{obj_class} {conf:.4f}', (x1,y1), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255))
    
    print(x, y, width, height, conf, obj_class)
    
cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()

129.5 119.0 257.0 96.0 0.8729892373085022 Scissors
117.0 316.5 230.0 123.0 0.7074464559555054 Paper
128.0 502.5 254.0 109.0 0.4878521263599396 Paper


In [4]:
from inference_sdk import InferenceHTTPClient
CLIENT1 = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key = os.getenv('ROBOFLOW_API_KEY')
)
CLIENT1

In [14]:
result1 = CLIENT1.infer('./mnist_train/000002.jpg', model_id="numbers-rioqd/2")
result1

{'inference_id': '5555b1d2-9474-4d0e-b816-acbd2541f478',
 'time': 0.03505561299971305,
 'image': {'width': 416, 'height': 416},
 'predictions': [{'x': 264.0,
   'y': 313.5,
   'width': 28.0,
   'height': 43.0,
   'confidence': 0.9781924486160278,
   'class': '3',
   'class_id': 3,
   'detection_id': '58eeb0ce-4f88-42e8-81cf-7a3bbb78cf8e'},
  {'x': 377.5,
   'y': 330.0,
   'width': 29.0,
   'height': 36.0,
   'confidence': 0.9623201489448547,
   'class': '3',
   'class_id': 3,
   'detection_id': 'ceb0396b-6373-4472-90c1-6084ef096d1d'},
  {'x': 195.0,
   'y': 279.0,
   'width': 28.0,
   'height': 44.0,
   'confidence': 0.9390106201171875,
   'class': '6',
   'class_id': 6,
   'detection_id': 'd24b00de-27a5-47d6-bdab-93dc3a977d2c'},
  {'x': 196.5,
   'y': 66.0,
   'width': 13.0,
   'height': 14.0,
   'confidence': 0.7679268717765808,
   'class': '2',
   'class_id': 2,
   'detection_id': '8228d540-119f-4c74-8e21-edc0730a2c25'},
  {'x': 299.5,
   'y': 24.5,
   'width': 13.0,
   'height': 19

# model 가져오기

In [9]:
from roboflow import Roboflow
import cv2
rf = Roboflow(api_key=os.getenv('ROBOFLOW_API_KEY'))
project = rf.workspace().project('numbers-yt0h2')
numbers_model = project.version(2).model
numbers_model

loading Roboflow workspace...
loading Roboflow project...


In [18]:
import cv2
img = cv2.imread('./mnist_train/000002.jpg')

filename = './mnist_train/000002.jpg'
result = numbers_model.predict(filename, confidence=40, overlap=30)
result_json = result.json()
print(result_json)

for pred in result_json['predictions']:
    x = pred['x']
    y = pred['y']
    width = pred['width']
    height = pred['height']
    conf = pred['confidence']
    cls = pred['class']
    print(x, y, width, height, f'{cls}{conf:.4f}')
    
cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()

{'predictions': [{'x': 109, 'y': 234, 'width': 7, 'height': 8, 'confidence': 0.5137761831283569, 'class': '2', 'class_id': 2, 'detection_id': 'b9453b38-89ce-47db-b2b8-0279d9ed49ea', 'image_path': './mnist_train/000002.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 76, 'y': 126, 'width': 25, 'height': 30, 'confidence': 0.49453362822532654, 'class': '8', 'class_id': 8, 'detection_id': '39761633-6698-427a-aacd-05a1514ee15d', 'image_path': './mnist_train/000002.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 234, 'y': 30, 'width': 45, 'height': 46, 'confidence': 0.4825494885444641, 'class': '3', 'class_id': 3, 'detection_id': 'e63e37ea-e6d7-4d63-849f-f9f1e9fffd4d', 'image_path': './mnist_train/000002.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 233, 'y': 30, 'width': 47, 'height': 44, 'confidence': 0.4006369411945343, 'class': '2', 'class_id': 2, 'detection_id': 'dbe3d026-6131-44aa-9869-f4125da8047a', 'image_path': './mnist_train/000002.jpg', 'prediction_type': 

KeyboardInterrupt: 

In [21]:
filename = 'mnist_train/000002.jpg'
result = numbers_model.predict(filename, confidence=40, overlap=30)
result_json = result.json()
#print(result_json)

for pred in result_json['predictions']:
    x = pred['x']
    y = pred['y']
    width = pred['width']
    height = pred['height']
    conf = pred['confidence']
    cls = pred['class']
    print(x, y, width, height, f'{cls} {conf:.4f}')
    
    x1, y1 = int(x-width/2), int(y-height/2)
    x2, y2 = int(x+width/2), int(y+height/2)
    
    cv2.rectangle(img, (x1,y1), (x2, y2), (0,0,255), 2)
    cv2.putText(img, f'{cls} {conf:.4f}', (x1,y1), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255))

cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()

109 234 7 8 2 0.5138
76 126 25 30 8 0.4945
234 30 45 46 3 0.4825
233 30 47 44 2 0.4006


KeyboardInterrupt: 